# Estudio de Repitibilidad y Reproducibilidad

### Rango de cada parte del equipo
$$R = X_{max} - X_{min}$$

In [103]:
def rango(mediciones):
    rango_list = list(range(0, len(mediciones)))
    for i in range(len(rango_list)):
        rango_list[i] = max(mediciones[i])-min(mediciones[i])
    return rango_list

In [104]:
rango([[300.4,305.6,300.8,306.1,305],[300.1,299.9,300,300,299.9],[300.5,300.4,300.4,300.5,300.6]])

[5.7000000000000455, 0.20000000000004547, 0.20000000000004547]

### Rango promedio por operador
$$ R = \frac {1}{n} \sum_{i=1}^n R_i $$

In [40]:
def rango_mean(mediciones):
    sum_r = sum(rango(mediciones))
    r_mean = 1/len(mediciones)
    result = r_mean * sum_r
    return result

In [41]:
rango_mean([[300.4,305.6,300.8,306.1,305],[300.1,299.9,300,300,299.9],[300.5,300.4,300.4,300.5,300.6]])

2.0333333333333785

### Rangos promedios totales



In [42]:
import pandas as pd

In [53]:
data = pd.read_excel("Libro1.xlsx")
data.head()

,operador,omega1,omega2,omega3
0,A,300.4,300.1,300.5
1,A,305.6,299.9,300.4
2,A,300.8,300.0,300.4
3,A,306.1,300.0,300.5
4,A,305.0,299.9,300.6


In [87]:
ops = data["operador"].unique().tolist()

In [88]:
ops

['A', 'B', 'C']

In [112]:
rango([[300.4, 305.6, 300.8, 306.1, 305.0]])

[5.7000000000000455]

In [109]:
data[data["operador"]=="A"]["omega1"].tolist()

[300.4, 305.6, 300.8, 306.1, 305.0]